This is to demonstrate how to relaunch failed jobs given the name of the deployment (e.g. ```TibetA_165```).

In [1]:
import hyp3_sdk

# Define job-name
job_name = 'WestCoastN_13'

# Specify deployment URL (uncomment one of the lines below as appropriate)
#deploy_url = 'https://hyp3-tibet.asf.alaska.edu' #for Tibet
deploy_url = 'https://hyp3-isce.asf.alaska.edu' #for access

In [2]:
# parse jobs
hyp3_isce = hyp3_sdk.HyP3(deploy_url)
pending_jobs = hyp3_isce.find_jobs(status_code='PENDING') + \
               hyp3_isce.find_jobs(status_code='RUNNING') + \
               hyp3_isce.find_jobs(status_code='SUCCEEDED')

failed_jobs = hyp3_isce.find_jobs(status_code='FAILED', name=job_name)

In [113]:
# restructure dictionaries to make parsing easier
pending_jobs = [{'name': r.name,
              'job_type': r.job_type,
              'job_parameters': {'granules': r.job_parameters['granules'],
                                 'secondary_granules': r.job_parameters['secondary_granules']}} 
             # NOTE THERE IS AN INDEX - this is to submit only a subset of Jobs
             for r in pending_jobs]

failed_jobs = [{'name': r.name,
              'job_type': r.job_type,
              'logs': r.logs,
              'job_parameters': {'granules': r.job_parameters['granules'],
                                 'secondary_granules': r.job_parameters['secondary_granules']}} 
             # NOTE THERE IS AN INDEX - this is to submit only a subset of Jobs
             for r in failed_jobs]

Ensure only unprocessed failed scenes are passed

In [114]:
# Remove duplicate failed jobs
done = []
result = []
for d in failed_jobs:
    if d['job_parameters'] not in done:
        done.append(d['job_parameters'])  # note it down for further iterations
        result.append(d)
print("We are removing {} duplicate failed jobs for submission".format(len(failed_jobs) - len(result)))
failed_jobs = result

# Remove duplicate pending/running/succeeded jobs
done = []
result = []
for d in pending_jobs:
    if d['job_parameters'] not in done:
        done.append(d['job_parameters'])  # note it down for further iterations
for d in failed_jobs:
    if d['job_parameters'] not in done:
        result.append(d)
print("We are removing {} failed jobs for submission that are being/have been processed".format(len(failed_jobs) - len(result)))
failed_jobs = result

We are removing 0 duplicate failed jobs for submission
We are removing 71 failed jobs for submission that are being/have been processed


Print basic info/logs for each failed job that needs to be rerun

In [115]:
# query failed jobs and print URLs for log output
for job in failed_jobs:
    print('Job log:', job['logs'])

Job log: ['https://hyp3-isce-contentbucket-4xpualmsjg98.s3.us-west-2.amazonaws.com/80139ac5-79bd-4e44-88b3-84ab55398d3b/80139ac5-79bd-4e44-88b3-84ab55398d3b.log']
Job log: ['https://hyp3-isce-contentbucket-4xpualmsjg98.s3.us-west-2.amazonaws.com/6c09d14b-4073-420e-8c4a-2bd47a3f69a7/6c09d14b-4073-420e-8c4a-2bd47a3f69a7.log']
Job log: ['https://hyp3-isce-contentbucket-4xpualmsjg98.s3.us-west-2.amazonaws.com/24b336c9-cd8d-41c1-b53d-e014725bfd12/24b336c9-cd8d-41c1-b53d-e014725bfd12.log']
Job log: ['https://hyp3-isce-contentbucket-4xpualmsjg98.s3.us-west-2.amazonaws.com/35f767d1-2bf8-4f8a-b7a6-20df1d420f1d/35f767d1-2bf8-4f8a-b7a6-20df1d420f1d.log']
Job log: ['https://hyp3-isce-contentbucket-4xpualmsjg98.s3.us-west-2.amazonaws.com/e4c0fa2e-3feb-4087-b07b-eefb1f5a72ca/e4c0fa2e-3feb-4087-b07b-eefb1f5a72ca.log']
Job log: ['https://hyp3-isce-contentbucket-4xpualmsjg98.s3.us-west-2.amazonaws.com/89fe3dd2-b732-414c-8c64-b050831c77ae/89fe3dd2-b732-414c-8c64-b050831c77ae.log']
Job log: ['https://hyp

In [109]:
failed_jobs[0:2]

[{'name': 'WestCoastN_13',
  'job_type': 'INSAR_ISCE',
  'job_parameters': {'granules': ['S1B_IW_SLC__1SDV_20180524T142030_20180524T142057_011064_01446D_6C91'],
   'secondary_granules': ['S1B_IW_SLC__1SDV_20180430T142029_20180430T142056_010714_013915_7F01']}},
 {'name': 'WestCoastN_13',
  'job_type': 'INSAR_ISCE',
  'job_parameters': {'granules': ['S1B_IW_SLC__1SDV_20180524T142030_20180524T142057_011064_01446D_6C91'],
   'secondary_granules': ['S1B_IW_SLC__1SDV_20180418T142028_20180418T142055_010539_01337A_D0CA']}}]

Re-query the job to check progress

In [124]:
prepared_jobs = []
for d in failed_jobs:
    d.pop('logs', None)
    prepared_jobs.append(d)

In [130]:
#UNCOMMENT TO SUBMIT
#submitted_jobs = hyp3_sdk.Batch()
#for batch in hyp3_sdk.util.chunk(prepared_jobs):
#    submitted_jobs += hyp3_isce.submit_prepared_jobs(batch)

Re-query the job to check progress

In [131]:
jobs_rerun = hyp3_isce.find_jobs(name=job_name)
print(jobs_rerun)

2639 HyP3 Jobs: 1236 succeeded, 737 failed, 270 running, 396 pending.


Track job progress

In [132]:
# parse jobs
updated_jobs = hyp3_isce.find_jobs(status_code='PENDING') + \
               hyp3_isce.find_jobs(status_code='RUNNING') + \
               hyp3_isce.find_jobs(status_code='SUCCEEDED')

# restructure dictionaries to make parsing easier
updated_jobs = [{'name': r.name,
              'job_type': r.job_type,
              'job_parameters': {'granules': r.job_parameters['granules'],
                                 'secondary_granules': r.job_parameters['secondary_granules']}} 
             # NOTE THERE IS AN INDEX - this is to submit only a subset of Jobs
             for r in updated_jobs]

# Find jobs that failed again
done = []
result = []
for d in updated_jobs:
    if d['job_parameters'] not in done:
        done.append(d['job_parameters'])  # note it down for further iterations
for d in failed_jobs:
    if d['job_parameters'] not in done:
        result.append(d)
print("There are {} jobs that failed again after reprocessing".format(len(result)))

There are 0 jobs that failed again after reprocessing


In [133]:
print("See each failed job printed below:")
for d in result:
    print(d)

See each failed job printed below:
